In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0,'..')

In [46]:
from mytools.tools import *

from food.paths import *
from food.psql import *

import pandas as pd
import numpy as np

# !nbdev_build_lib
from pathlib import Path

from mytools.visual import get_text_clip,search_clip,series2tensor

In [47]:
data_path = Path('../food_dataset')

In [48]:
input_food = (pd.read_csv(data_path/'input_food.csv').set_index('fdc_id')[['gram_weight','sr_description']]
              .rename(columns = {'sr_description':'input','gram_weight':'input_weight'})
              .sort_values('input_weight'))

In [49]:
food = pd.read_csv(data_path/'food.csv')

In [50]:
food = food[food['data_type']=='survey_fndds_food'].set_index('fdc_id')[['description']]

In [51]:
food = food.join(input_food)

In [52]:
food = food[~food['input'].isin(['Beverages, water, tap, drinking',
                                 'Salt, table, iodized'])]

In [53]:
def func(g):
    g = g.sort_values('input_weight')
    g['cumsum'] = g['input_weight'].cumsum()/g['input_weight'].sum()
    g = g.sort_values('cumsum',ascending = False)
    
    g = g[g['cumsum']>0.05]
    return g

In [54]:
food = food.groupby('fdc_id').apply(func)

In [55]:
food = food.sort_values('input_weight',ascending = False)

In [56]:
food.index = [i[1] for i in food.index]

In [57]:
f = food.reset_index().groupby('index').agg({'description':'first','input':lambda l:'; '.join(l)})

In [61]:
df = f.apply(lambda ff:f'"{ff["description"]}" made of {ff["input"]}',axis=1)

In [62]:
df = df.reset_index(name='text').rename(columns = {'index':'food_id'})

In [63]:
df

,food_id,text
0,1097510,"""Milk, human"" made of Milk, human, mature, fluid"
1,1097511,"""Milk, NFS"" made of Milk, reduced fat, fluid, ..."
2,1097512,"""Milk, whole"" made of Milk, whole, 3.25% milkf..."
3,1097513,"""Milk, low sodium, whole"" made of Milk, low so..."
4,1097514,"""Milk, calcium fortified, whole"" made of Milk,..."
...,...,...
7077,1104588,"""Tomatoes as ingredient in omelet"" made of Tom..."
7078,1104589,"""Other vegetables as ingredient in omelet"" mad..."
7079,1104590,"""Vegetables as ingredient in curry"" made of On..."
7080,1104591,"""Sauce as ingredient in hamburgers"" made of Ca..."


In [64]:
df['version'] = 2

In [65]:
df.to_sql('foods_prompted',engine,if_exists = 'append',index = False,schema = 'food')

82

In [66]:
pd.read_sql('select * from food.foods_prompted',engine)

,id,food_id,text,version,clip
0,1867,1099376,"the photo of ""Meat loaf made with chicken or ...",1,"[-0.009547241, 0.027974756, -0.0008684345, 0.0..."
1,1868,1099377,"the photo of ""Clams, stuffed"" dish, which is ...",1,"[0.034463722, 0.006951217, -0.030041857, 0.020..."
2,1869,1099378,"the photo of ""Codfish ball or cake"" dish, whi...",1,"[0.009532085, 0.012929418, 0.033163667, 0.0550..."
3,1870,1099379,"the photo of ""Crab cake"" dish, which is made ...",1,"[-0.0019750579, 0.011804303, -0.019615391, 0.0..."
4,1871,1099380,"the photo of ""Fish cake or patty, NS as to fi...",1,"[0.0056863017, 0.0066959057, 0.015637038, 0.03..."
...,...,...,...,...,...
14159,7078,1104588,"the photo of ""Tomatoes as ingredient in omele...",1,"[0.07593674, 0.023695257, 0.014061779, 0.03602..."
14160,7079,1104589,"the photo of ""Other vegetables as ingredient ...",1,"[0.07699164, 0.014432207, 0.022694767, 0.04846..."
14161,7080,1104590,"the photo of ""Vegetables as ingredient in cur...",1,"[0.0098340195, 0.00041265108, 0.0053099045, 0...."
14162,7081,1104591,"the photo of ""Sauce as ingredient in hamburge...",1,"[0.019054852, 0.027971897, 0.029259693, -0.009..."


In [86]:
q = """select p.clip, p.text, p.version,f.description,f.category 
       from food.foods_prompted as p
       join food.foods as f on (p.food_id = f.id)
       where p.clip is not null
       """

In [87]:
foods = pd.read_sql(q,engine)

In [77]:
foods_old = pd.read_sql('select * from food.foods where clip is not null',engine)

In [79]:
foods

,clip,text,version,description
0,"[-0.009547241, 0.027974756, -0.0008684345, 0.0...","the photo of ""Meat loaf made with chicken or ...",1,"Meat loaf made with chicken or turkey, with to..."
1,"[0.034463722, 0.006951217, -0.030041857, 0.020...","the photo of ""Clams, stuffed"" dish, which is ...",1,"Clams, stuffed"
2,"[0.009532085, 0.012929418, 0.033163667, 0.0550...","the photo of ""Codfish ball or cake"" dish, whi...",1,Codfish ball or cake
3,"[-0.0019750579, 0.011804303, -0.019615391, 0.0...","the photo of ""Crab cake"" dish, which is made ...",1,Crab cake
4,"[0.0056863017, 0.0066959057, 0.015637038, 0.03...","the photo of ""Fish cake or patty, NS as to fi...",1,"Fish cake or patty, NS as to fish"
...,...,...,...,...
9093,"[0.029202031, 0.0440451, -0.008807819, 0.04436...","""Chicken or turkey, potatoes, and vegetables i...",2,"Chicken or turkey, potatoes, and vegetables in..."
9094,"[0.030916419, 0.040611614, -0.009172419, 0.048...","""Chicken or turkey, potatoes, and vegetables e...",2,"Chicken or turkey, potatoes, and vegetables ex..."
9095,"[0.017849905, 0.052032243, -0.011163072, 0.035...","""Chicken or turkey, potatoes, and vegetables i...",2,"Chicken or turkey, potatoes, and vegetables in..."
9096,"[0.027764209, 0.048447695, -0.007417399, 0.044...","""Chicken or turkey, potatoes, and vegetables e...",2,"Chicken or turkey, potatoes, and vegetables ex..."


In [80]:
food_clips = series2tensor(foods['clip'])

In [81]:
food_clips_old = series2tensor(foods_old['clip'])

In [82]:
url = 'https://img-global.cpcdn.com/recipes/2e5de88897d95c0c/400x400cq70/photo.jpg'

In [83]:
r = search_clip(url,foods,food_clips)[1];r

,index,clip,text,version,description,score
0,4148,"[0.0111375665, 0.032855168, 0.016608622, 0.034...","the photo of ""Barley, fat added"" dish, which ...",1,"Barley, fat added",0.311238


In [84]:
r['text'].iloc[0]

' the photo of "Barley, fat added" dish, which is made of the following: Barley, pearled, cooked'

In [85]:
search_clip(url,foods_old,food_clips_old)[1]

,index,id,description,category,energy,protein,carb,fat,clip,score
0,4047,1101537,"Buckwheat groats, no added fat","Pasta, noodles, cooked grains",92.0,3.37,19.86,0.62,"[0.031248957, 0.014652966, 0.029392239, 0.0195...",0.272619


In [43]:
search_clip

<function food.tools.search_clip(url, foods, food_clips, head=1)>

In [29]:
engine.execute("drop table food.foods_prompted")

ProgrammingError: (psycopg2.errors.UndefinedTable) table "foods_prompted" does not exist

[SQL: drop table food.foods_prompted]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [30]:
df

,food_id,text,version
0,1097510,"the photo of ""Milk, human"" dish, which is mad...",1
1,1097511,"the photo of ""Milk, NFS"" dish, which is made ...",1
2,1097512,"the photo of ""Milk, whole"" dish, which is mad...",1
3,1097513,"the photo of ""Milk, low sodium, whole"" dish, ...",1
4,1097514,"the photo of ""Milk, calcium fortified, whole""...",1
...,...,...,...
7077,1104588,"the photo of ""Tomatoes as ingredient in omele...",1
7078,1104589,"the photo of ""Other vegetables as ingredient ...",1
7079,1104590,"the photo of ""Vegetables as ingredient in cur...",1
7080,1104591,"the photo of ""Sauce as ingredient in hamburge...",1


In [7]:
get_text_clip('dogs')

tensor([ 1.6915e-02,  1.2465e-02,  8.9097e-03, -2.9618e-02,  1.7291e-02,
         1.7569e-03,  1.9857e-02,  8.7645e-03, -9.6193e-03, -3.8230e-02,
        -1.6027e-02, -1.1519e-02,  1.5489e-02,  1.2435e-02, -1.1473e-02,
         9.1915e-03, -4.4956e-03,  1.3724e-02, -1.2408e-02,  8.3505e-03,
        -2.4924e-02, -1.0489e-02,  1.8021e-02,  9.3614e-04,  1.5869e-02,
         8.7736e-04, -2.8382e-03, -1.2125e-02, -4.0839e-03,  1.0921e-03,
        -2.2746e-02,  8.6543e-03, -1.8522e-02, -4.0211e-02, -1.6811e-02,
        -7.1129e-03, -1.7050e-02, -1.8504e-03, -2.1892e-02,  5.4940e-03,
         1.2249e-02,  8.0643e-03, -5.4576e-03, -2.4247e-03,  8.2767e-03,
        -9.8407e-03,  1.0315e-03,  1.2344e-02,  1.1869e-02, -1.3152e-02,
        -3.3823e-03,  1.6768e-03,  2.2077e-02,  2.5708e-02, -1.2388e-02,
         1.6520e-02, -1.7282e-03,  1.7294e-02, -6.7715e-03,  5.1089e-03,
         1.1791e-03,  1.3683e-02,  3.0604e-02, -2.8036e-02, -7.3349e-03,
         7.9450e-03, -4.9110e-03,  2.2022e-03,  1.3